<a href="https://colab.research.google.com/github/ramya2886/fargees/blob/main/chatbot_using_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow numpy

In [ ]:
# Sample conversation data
conversations = [
    ("Hi", "Hello! How can I help you?"),
    ("How are you?", "I am fine, thank you!"),
    ("What is your name?", "I am a chatbot."),
    ("Bye", "Goodbye! Have a nice day."),
    ("Tell me a joke", "Why did the computer go to the doctor? Because it caught a virus!")
]

# Separate inputs and outputs
questions = [q[0] for q in conversations]
answers = [a[1] for a in conversations]
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizer
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(questions + answers)

vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
question_seq = tokenizer.texts_to_sequences(questions)
answer_seq = tokenizer.texts_to_sequences(answers)

# Pad sequences
max_len = max(max(len(seq) for seq in question_seq),
              max(len(seq) for seq in answer_seq))

question_seq = pad_sequences(question_seq, maxlen=max_len, padding='post')
answer_seq = pad_sequences(answer_seq, maxlen=max_len, padding='post')
# Decoder target data is shifted by one timestep
decoder_target_seq = np.zeros_like(answer_seq)
decoder_target_seq[:, :-1] = answer_seq[:, 1:]
decoder_target_seq[:, -1] = 0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

embedding_dim = 50
lstm_units = 100

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len,))
decoder_embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define and compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()
model.fit([question_seq, answer_seq], decoder_target_seq,
          batch_size=2,
          epochs=500)

# Define inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_h_input = Input(shape=(lstm_units,))
decoder_state_c_input = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_h_input, decoder_state_c_input]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_layer(decoder_inputs),
                                                 initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def respond(input_text):
    seq = tokenizer.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=max_len, padding='post')

    # Encode input and get initial states
    states_value = encoder_model.predict(seq)

    target_seq = np.zeros((1, 1))
    output_sentence = []

    for _ in range(max_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:  # Stop if start token (or unknown, depending on your setup)
            break

        output_sentence.append(tokenizer.index_word[sampled_token_index])

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(output_sentence)

print("Chatbot (type 'exit' to quit)")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = respond(user_input)
    print(f"Chatbot: {response}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 13, 50)    │      1,900 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 13, 50)    │      1,900 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 100),     │     60,400 │ embedding_2[0][0] │
│                     │ (None, 100),      │            │                   │
│                     │ (None, 100)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 13, 100), │     60,400 │ embedding_3[0][0… │
│                     │ (None, 100),      │            │ lstm_2[0][1],     │
│                     │ (None, 100)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 13, 38)    │      3,838 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 128,438 (501.71 KB)

 Trainable params: 128,438 (501.71 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 3.6253
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 3.5588
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 3.4492
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.1706
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.5220
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.8625
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.7686
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.6161
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.1272
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.0104
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.9099
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.4954
Epoch 13/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.5725
Epoch 14/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1.3672
Epoch 15/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.4475
Epoch 16/500
3/3 ━━